# Testing the lambda function code locally

In [1]:
!pip install boto3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 10.9 MB/s eta 0:00:00


### Saving an access key and secret key in an environment variable
---

In [3]:
import os
from IPython.display import clear_output

def set_environment_variable_values():
  ACCESS_KEY = input("Please enter the AWS access key: ")
  SECRET_ACCESS_KEY = input("Please enter the AWS secret access key: ")
  BUCKET_NAME = input("Please enter the name of the bucket in S3: ")
  os.environ['ACCESS_KEY'] = ACCESS_KEY
  os.environ['SECRET_ACCESS_KEY'] = SECRET_ACCESS_KEY
  os.environ['BUCKET_NAME'] = BUCKET_NAME
  clear_output()
  return None

set_environment_variable_values()


### Creating a connection to the S3 bucket
---

In [6]:
import boto3

def get_S3_client():
	resource = boto3.client(
     "s3",
		aws_access_key_id = os.environ.get('ACCESS_KEY'),
		aws_secret_access_key = os.environ.get('SECRET_ACCESS_KEY')
	)
	return resource

s3_client = get_S3_client()

### Defining the functions

---
Defining get_file(filename) and save_a_copy(filedata, filename) functions that will be used to test the lambda_handler(event, context) function. Defining the lambda_handler(event, context) function copied from AWS Lambda. Some code in lambda_handler is hashed out or replaced with get_file(filename) and save_a_copy(filedata, filename) function calls in order to be able to test it locally.

In [28]:
import pandas as pd
import io
import os
import json
import csv
#from functions import add_school_data_to_bucket, show_schools_data_in_bucket

def get_file(filename):
  # get the file from the bucket
  file_object = s3_client.get_object(Bucket=os.environ.get('BUCKET_NAME'), Key=filename)
  # convert the file object to a text-based csv file then read the file contents into a table using the pandas read_csv function
  data_file = io.BytesIO(file_object['Body'].read())
  data = pd.read_csv(data_file)
  return data

def save_a_copy(filedata, filename):
  new_data = filedata.copy()
  file_object =  io.StringIO()
  pd.DataFrame(new_data).to_csv(file_object, index=False)
  response = s3_client.put_object(Bucket=os.environ.get('BUCKET_NAME'), Body=file_object.getvalue(), Key=filename)
  return response

def lambda_handler(event, context):
    global message, return_data, statuscode
    filename = "schools_list.csv"
    client = boto3.client('s3')
    if event["httpMethod"] == "POST":
        if "body" in event.keys():
            request = event["body"]
            if type(request) is not dict:
                request = json.loads(request)
            if request is not None and "data" in request.keys():
                data = request["data"]
                if len(data) == 0:
                    message, return_data = "Please enter a valid data", []
                    statuscode = 404
                else:
                    # the line below is different from the original Lambda function
                    return_data = save_a_copy(data,'schools_list.csv')
                    #message, return_data = add_school_data_to_bucket(client, filename, data)
                    message="Yay! Success"
                    statuscode = 200
            else:
                message, return_data = "Error in the POST request occured", []
                statuscode = 404
    elif event["httpMethod"] == "GET":
       # the line below is different from the original Lambda function
       return_data = get_file('schools_list.csv')
       message="Yay! Success"
       statuscode = 200
       #message, return_data = show_schools_data_in_bucket(client, filename)
    else:
        message, return_data = "Error occured", []
        statuscode = 404
    return {'statusCode': statuscode,
            'headers': {'Content-Type': 'application/json',
                        'Access-Control-Allow-Headers': 'Content-Type,X-Api-Key',
                        'Access-Control-Allow-Methods': 'POST',
                        'Access-Control-Allow-Origin': '*'},
            #'body': json.dumps({"message": message, "data": return_data})
            'body': {"message": message, "data": return_data}
            }

### Testing the lambda_handler(event, context) function using POST method
---
When POST method is used in the test, we check if the code works using the save_a_copy(filedata, filename) function.

In [29]:
event1 = {
    "httpMethod": "POST",
    "body": {
    "data": [
        [
            "Bannockburn Primary School & Nursery",
            51.4869172,
            0.1015561
        ],
        [
            "St Margaret Clitherow Primary School",
            51.50103289999999,
            0.1132992
        ]
    ]
}
}

lambda_handler(event1, None)

{'statusCode': 200,
 'headers': {'Content-Type': 'application/json',
  'Access-Control-Allow-Headers': 'Content-Type,X-Api-Key',
  'Access-Control-Allow-Methods': 'POST',
  'Access-Control-Allow-Origin': '*'},
 'body': {'message': 'Yay! Success',
  'data': {'ResponseMetadata': {'RequestId': '0G6Z7KABES8C3PHB',
    'HostId': 'KI1bRUMwiTR+ESlh3ZytrpGfdlSeAnmd6l9w+YTQHPD6ZkFjlFhooUrum3vUwiW8D4Rbjem2ImGoZPmL5Tt6Pw==',
    'HTTPStatusCode': 200,
    'HTTPHeaders': {'x-amz-id-2': 'KI1bRUMwiTR+ESlh3ZytrpGfdlSeAnmd6l9w+YTQHPD6ZkFjlFhooUrum3vUwiW8D4Rbjem2ImGoZPmL5Tt6Pw==',
     'x-amz-request-id': '0G6Z7KABES8C3PHB',
     'date': 'Mon, 11 Mar 2024 16:47:32 GMT',
     'x-amz-server-side-encryption': 'AES256',
     'etag': '"ddb9933f885c2a08fb1a37b457cb3b00"',
     'server': 'AmazonS3',
     'content-length': '0'},
    'RetryAttempts': 0},
   'ETag': '"ddb9933f885c2a08fb1a37b457cb3b00"',
   'ServerSideEncryption': 'AES256'}}}

### Testing the lambda_handler(event, context) function using GET method
---
When GET method is used in the test, we check if the code works using the get_file(filename) function.

In [30]:
event2 = {
    "httpMethod": "GET",
}

lambda_handler(event2, None)

{'statusCode': 200,
 'headers': {'Content-Type': 'application/json',
  'Access-Control-Allow-Headers': 'Content-Type,X-Api-Key',
  'Access-Control-Allow-Methods': 'POST',
  'Access-Control-Allow-Origin': '*'},
 'body': {'message': 'Yay! Success',
  'data':                                       0          1         2
  0  Bannockburn Primary School & Nursery  51.486917  0.101556
  1  St Margaret Clitherow Primary School  51.501033  0.113299}}